In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
from sklearn.metrics import accuracy_score
from keras import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense, CuDNNLSTM, LSTM
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


In [2]:
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')
DIRECTORY = '/content/gdrive/MyDrive/Colab Notebooks/'
DATA = DIRECTORY + 'vehicle_data/' 

/
Mounted at /content/gdrive


In [3]:
df = pd.read_csv(DATA + 'records_all_csv.csv')
df.head()

,record_id,timezone,created,modified,occurred_from,occurred_from_hour,occurred_to,lat,lon,location_text,...,weather,light,Severity,driverIncidentDetails_id,Main cause,Description,Collision-type,Email of Encoder,Reporting Agency,Location Approximate
0,a913cced-1b3d-41ed-b5e6-52ba87816c49,UTC,2021-10-15 08:49:04,2021-10-15 08:49:04,2016-09-15 04:55:52,4,2021-10-15 08:41:52,8.503039,124.642709,Rodolfo N. Pelaez Boulevard - San Pedro St. Ju...,...,clear-day,dawn,Property Damage,7e5037c0-8383-434c-a40d-5ff2600df7f9,Human error,MC vs. PUJ,Side-swipe,rta.cdo@gmail.com,Other,Yes
1,cdf2e78c-a345-4ecb-9299-49aa93eacc1c,UTC,2021-10-15 08:38:18,2021-10-15 08:50:19,2016-12-11 03:32:50,3,2021-10-15 08:32:50,8.496025,124.625377,"Liempo King, Mother of Perpetual Help Avenue, ...",...,clear-day,day,Injury,37c04e5d-238b-4cd1-bbdd-2b0ea88a38a6,Human error,NaN,Side-swipe,rta.cdo@gmail.com,Other,Yes
2,04660c01-6070-4e6e-850d-f2aeed151eb7,UTC,2019-01-01 12:55:33,2019-01-01 12:55:33,2019-01-01 07:01:36,7,2019-01-01 07:01:36,14.599614,121.049595,"Shell, Ortigas Avenue, Greenhills, West Crame,...",...,partly-cloudy-day,day,Property Damage,d271e8b3-155a-41d6-9226-02c99e97e782,Human error,NaN,Side-swipe,NaN,MMDA Metrobase,NaN
3,b1bb4e9e-981f-4101-87f7-86ac494c3719,UTC,2019-01-01 13:01:52,2019-01-01 13:01:52,2019-01-01 07:06:27,7,2019-01-01 07:06:27,14.536532,120.988870,"Diosdado Macapagal Boulevard, Metropolitan Par...",...,partly-cloudy-day,day,Property Damage,5a9bafeb-bf22-4d29-910d-19c349353d9b,Human error,NaN,Side-swipe,NaN,MMDA Metrobase,NaN
4,d12db962-0464-4944-b376-f993d272ee2b,UTC,2019-01-01 13:03:27,2019-01-01 13:03:27,2019-01-01 08:23:46,8,2019-01-01 08:23:46,14.631424,121.074142,"Katipunan Flyover, Escopa III, Project 4, Dist...",...,partly-cloudy-day,day,Property Damage,3b7fcdf1-c853-4699-95d2-49936f56875d,Human error,NaN,Side-swipe,NaN,MMDA Metrobase,NaN


In [4]:
df = df[['lat', 'lon', 'weather', 'light', 'Main cause', 'Collision-type', 'occurred_from_hour', 'Severity' ]]
df = df.rename(columns={'Main cause': 'main_cause', 'Collision-type': 'collision_type', 'occurred_from_hour' : 'occurred_from', 'Severity' : 'severity'})
df = df.iloc[:-1]

In [5]:
df.shape

(11265, 8)

In [6]:
df = df.dropna()
df.reset_index(drop=True)
df.shape

(8798, 8)

In [7]:
df = df.iloc[:-1]
df

,lat,lon,weather,light,main_cause,collision_type,occurred_from,severity
0,8.503039,124.642709,clear-day,dawn,Human error,Side-swipe,4,Property Damage
1,8.496025,124.625377,clear-day,day,Human error,Side-swipe,3,Injury
2,14.599614,121.049595,partly-cloudy-day,day,Human error,Side-swipe,7,Property Damage
3,14.536532,120.988870,partly-cloudy-day,day,Human error,Side-swipe,7,Property Damage
4,14.631424,121.074142,partly-cloudy-day,day,Human error,Side-swipe,8,Property Damage
...,...,...,...,...,...,...,...,...
11059,8.482878,124.630666,clear-day,day,Human error,Side-swipe,0,Property Damage
11060,8.504206,124.612920,clear-day,day,Human error,Side-swipe,1,Property Damage
11061,8.488735,124.659805,clear-day,day,Human error,Side-swipe,2,Property Damage
11062,8.503028,124.642709,clear-day,day,Human error,Side-swipe,5,Property Damage


In [ ]:
df.dtypes

lat               float64
lon               float64
weather            object
light              object
main_cause         object
collision_type     object
occurred_from       int64
severity           object
dtype: object

In [10]:
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

In [11]:
X

array([[8.503039024, 124.6427089, 'clear-day', ..., 'Human error',
        'Side-swipe', 4],
       [8.496025151, 124.6253765, 'clear-day', ..., 'Human error',
        'Side-swipe', 3],
       [14.59961448, 121.0495949, 'partly-cloudy-day', ...,
        'Human error', 'Side-swipe', 7],
       ...,
       [8.488735257, 124.6598053, 'clear-day', ..., 'Human error',
        'Side-swipe', 2],
       [8.503028413, 124.6427089, 'clear-day', ..., 'Human error',
        'Side-swipe', 5],
       [8.489382546, 124.6636248, 'clear-day', ..., 'Human error',
        'Side-swipe', 8]], dtype=object)

In [13]:
df = pd.DataFrame(X, columns = ['lat', 'lon', 'weather', 'light', 'main_cause', 'collision_type', 'occurred_from'])
df

,lat,lon,weather,light,main_cause,collision_type,occurred_from
0,8.503039,124.642709,clear-day,dawn,Human error,Side-swipe,4
1,8.496025,124.625377,clear-day,day,Human error,Side-swipe,3
2,14.599614,121.049595,partly-cloudy-day,day,Human error,Side-swipe,7
3,14.536532,120.98887,partly-cloudy-day,day,Human error,Side-swipe,7
4,14.631424,121.074142,partly-cloudy-day,day,Human error,Side-swipe,8
...,...,...,...,...,...,...,...
8792,8.482878,124.630666,clear-day,day,Human error,Side-swipe,0
8793,8.504206,124.61292,clear-day,day,Human error,Side-swipe,1
8794,8.488735,124.659805,clear-day,day,Human error,Side-swipe,2
8795,8.503028,124.642709,clear-day,day,Human error,Side-swipe,5


In [12]:
y

array(['Property Damage', 'Injury', 'Property Damage', ...,
       'Property Damage', 'Property Damage', 'Property Damage'],
      dtype=object)

In [ ]:
sorted(set(y))

In [14]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), ['weather', 'light', 'main_cause'])], remainder='passthrough')
X = np.array(ct.fit_transform(df))


In [ ]:
# assert 1 > 2

In [15]:
X.shape

(8797, 22)

In [16]:
X[0]

array([1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0,
       0.0, 1.0, 0.0, 0.0, 0.0, 8.503039024, 124.6427089, 'Side-swipe', 4],
      dtype=object)

In [17]:
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [20])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [18]:
X.shape

(8797, 33)

In [19]:
X[0]

array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0,
       0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0,
       1.0, 0.0, 0.0, 0.0, 8.503039024, 124.6427089, 4], dtype=object)

In [20]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)
print(y)

[5 3 5 ... 5 5 5]


In [21]:
print(set(y))

{0, 1, 2, 3, 4, 5}


In [22]:
total = X.shape[0]
total

8797

In [23]:
validation_len = (int) (total * 0.20)
validation_len

1759

In [24]:
percent_len = total-validation_len
percent_len

7038

In [25]:
X_validation = X[percent_len:]
X_train_test = X[:percent_len]
y_validation = y[percent_len:]
y_train_test = y[:percent_len]

In [26]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train_test, y_train_test, test_size = 0.15, random_state = 0)

In [27]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range=(0,1))
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
# from sklearn.preprocessing import StandardScaler
# sc = StandardScaler()
# X_train = sc.fit_transform(X_train)
# X_test = sc.transform(X_test)

In [28]:
X_test

array([[0.        , 0.        , 0.        , ..., 0.98544057, 0.02225984,
        0.04347826],
       [0.        , 0.        , 0.        , ..., 0.97394413, 0.02484406,
        0.47826087],
       [0.        , 0.        , 0.        , ..., 0.98338223, 0.0320095 ,
        0.30434783],
       ...,
       [0.        , 0.        , 0.        , ..., 0.97370114, 0.02464616,
        0.13043478],
       [0.        , 0.        , 0.        , ..., 0.98178022, 0.02555123,
        0.04347826],
       [0.        , 0.        , 0.        , ..., 0.99976385, 0.0270092 ,
        0.26086957]])

In [29]:
X_train.shape

(5982, 33)

In [30]:
X_train.shape[1:]

(33,)

In [31]:
y_train.shape

(5982,)

In [32]:
y_train

array([5, 5, 5, ..., 4, 5, 5])

In [33]:
set(y_train)

{1, 2, 3, 4, 5}

In [34]:
ann = tf.keras.models.Sequential()
ann.add(LSTM (32,  input_shape=(X_train.shape[1],1), return_sequences=True)) 
ann.add(Dropout(0.2))

ann.add(LSTM (32, return_sequences=True)) 
ann.add(Dropout(0.2))

# ann.add(LSTM (50, return_sequences=True)) 
# ann.add(Dropout(0.2))

ann.add(LSTM (32)) 
ann.add(Dropout(0.2))

# remove this
# ann.add(Dense(32))
# ann.add(Dropout(0.2))
# ann.add(Activation('relu'))

ann.add(tf.keras.layers.Dense(units=6, activation='softmax'))



In [35]:
from tensorflow.keras.optimizers import Adam
opt = Adam(lr=0.01)
ann.compile(optimizer = opt, loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
# from tensorflow.keras.optimizers import SGD
# opt = SGD(lr=0.2)
# ann.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [36]:
history = ann.fit(X_train, y_train, batch_size=8, epochs = 10, verbose=1)

Epoch 1/10
748/748 [==============================] - 47s 45ms/step - loss: 0.3286 - accuracy: 0.9159
Epoch 2/10
748/748 [==============================] - 28s 37ms/step - loss: 0.3204 - accuracy: 0.9168
Epoch 3/10
748/748 [==============================] - 29s 38ms/step - loss: 0.3146 - accuracy: 0.9168
Epoch 4/10
748/748 [==============================] - 28s 38ms/step - loss: 0.3121 - accuracy: 0.9168
Epoch 5/10
748/748 [==============================] - 28s 37ms/step - loss: 0.3135 - accuracy: 0.9168
Epoch 6/10
748/748 [==============================] - 28s 37ms/step - loss: 0.3127 - accuracy: 0.9168
Epoch 7/10
748/748 [==============================] - 29s 38ms/step - loss: 0.3129 - accuracy: 0.9168
Epoch 8/10
748/748 [==============================] - 28s 37ms/step - loss: 0.3122 - accuracy: 0.9168
Epoch 9/10
748/748 [==============================] - 27s 36ms/step - loss: 0.3124 - accuracy: 0.9168
Epoch 10/10
748/748 [==============================] - 27s 36ms/step - loss: 0.311

In [37]:
ann.evaluate(X_test,y_test, batch_size=8)

132/132 [==============================] - 2s 7ms/step - loss: 0.3048 - accuracy: 0.9148


[0.3047786056995392, 0.9147727489471436]

In [38]:
y_pred = ann.predict(X_test)
y_pred = np.amax(y_pred, axis=1)
y_pred

array([0.9161015 , 0.91610146, 0.9161015 , ..., 0.91610146, 0.9161015 ,
       0.91610146], dtype=float32)

In [39]:
y_pred = ann.predict(X_test, batch_size=100)
y_pred1D = y_pred.argmax(1)
print ('Accuracy on test data: ' + str(accuracy_score(y_test, y_pred1D)))

Accuracy on test data: 0.9147727272727273


In [ ]:
assert 1 > 2

AssertionError: ignored

VALIDATION

In [40]:
X_validation_test = sc.transform(X_validation)

In [41]:
y_pred_validation = ann.predict(X_validation_test, batch_size=100)
y_pred_validation1D = y_pred_validation.argmax(1)
print ('Accuracy on validation data: ' + str(accuracy_score(y_validation, y_pred_validation1D)))

Accuracy on validation data: 0.7970437748720864
